In [16]:
!pip3 install -U gensim

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 17.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 2.0 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme 

In [1]:
import pprint
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import open
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
import tempfile

In [2]:
#creating a document
document = "Human machine interface for lab abc computer applications"

In [3]:
#creating corpus
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [4]:
#defining stop words
stoplist = set('for a of the and to in'.split())

In [5]:
#iterating through the documents in the corpus and lowercasing the words and excluding the stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]
# list of lists of individual tokens of a document

In [6]:
#counting the qord frequencies
frequency = defaultdict(int)
for text in texts:
    #print(text)
    for token in text:
        #print(token)
        frequency[token] += 1

In [7]:
#only keeping words that appear more than once
processed_corpus = [[token for token in text if frequency[token]>1]for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [8]:
#creating a vocabulary of the words present in the corpus
#One need to feed the tokens got out of the corpus to the library function
#dictionary is then saved
vocabulary = corpora.Dictionary(processed_corpus)
vocabulary.save('deerwester.dict')
print(vocabulary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [9]:
#printing the vocabulary with tokens and their id
pprint.pprint(vocabulary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


In [10]:
#printing the (id,freq) of the words in a new sentence
#if a word is not present in the vocabulary no output regarding the same is given
new_doc = "Human Human computer computer computer eps interaction"
new_vec = vocabulary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 3), (1, 2), (8, 1)]


In [12]:
#creating the bag of words for our corpus
#this replaces the tokens of the documents with (id,freq) pairs
#frequency is with respect to the document and not the corpus
bow_corpus = [vocabulary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


In [13]:
#tfidf model simply replaces the frequency with the rarity parameter of the word in the corpus
# one can see that "minors" is rare than "system" in our corpus thus more weight is assigned to "minors"
tfidf = models.TfidfModel(bow_corpus)
words = "system minors".lower().split()
print(tfidf[vocabulary.doc2bow(words)])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


In [14]:
#creating a similarity query
index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=16)

In [15]:
#outputting the similarity of each of the document with th equery document
#note that we need to first convert the query doc to bag of words and apply the model to it
#then only we can find the similarity, kind of obvious but still mentioned it
query_doc = "system engineering".split()
query_bow = vocabulary.doc2bow(query_doc)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


In [16]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


In [17]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [18]:
#we can parse each document at a time as follows
class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            yield vocabulary.doc2bow(line.lower().split())

In [19]:
corpus_memory_friendly = MyCorpus()
print(corpus_memory_friendly)

In [20]:
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [21]:
# # collect statistics about all tokens
# dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/mycorpus.txt'))
# # remove stop words and words that appear only once
# stop_ids = [
#     dictionary.token2id[stopword]
#     for stopword in stoplist
#     if stopword in dictionary.token2id
# ]
# once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
# dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
# dictionary.compactify()  # remove gaps in id sequence after words that were removed
# print(dictionary)

In [22]:
#working with pretrained word2vec model based on google news
wv = api.load('word2vec-google-news-300')

2022-05-29 15:48:05,775 : INFO : loading projection weights from /Users/shri/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2022-05-29 15:48:49,833 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/shri/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-05-29T15:48:49.833364', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [23]:
#geeting thw vocabulary of the model
for index, word in enumerate(wv.index_to_key):
    if index==10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [26]:
#size of the embedding of the word king
wv['king'].shape

(300,)

In [33]:
#check if the word "cameroon" is present in the vocab or not
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("not found")


not found


In [29]:
#outputting the similarity measures of the given pair of words
pairs = [
    ('car', 'minivan'),  
    ('car', 'bicycle'),   
    ('car', 'airplane'),  
    ('car', 'cereal'),    
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


In [38]:
#training your own model
#we will feed our corpus to the class to get it preprocessed
#this will comvert to bag of words
#then we will feed this bow to model and will get a trained model for this corpus

In [39]:
class MyCorpus1:
    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            yield utils.simple_preprocess(line)

In [40]:
sentences = MyCorpus1()
model = gensim.models.Word2Vec(sentences=sentences)

2022-05-29 16:08:59,732 : INFO : collecting all words and their counts
2022-05-29 16:08:59,734 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-29 16:08:59,825 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-29 16:08:59,826 : INFO : Creating a fresh vocabulary
2022-05-29 16:08:59,835 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1750 unique words (25.068041827818366%% of original 6981, drops 5231)', 'datetime': '2022-05-29T16:08:59.835337', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-29 16:08:59,836 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 49335 word corpus (84.83801073049938%% of original 58152, drops 8817)', 'datetime': '2022-05-29T16:08:59.836257', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0

In [41]:
vec_king = model.wv['king']

In [42]:
print(vec_king)

[-0.02106759  0.06429333  0.01470954  0.0174621   0.01274141 -0.11490468
  0.04796187  0.12098181 -0.00810785 -0.0211865  -0.00792687 -0.07437152
  0.00718535  0.03965204  0.00878629  0.01540343 -0.00242962 -0.00237106
 -0.02652099 -0.0863167   0.05015018  0.01119104  0.01777221 -0.00151447
 -0.02835938  0.02660489 -0.02678897 -0.01507284 -0.03924191  0.01898382
  0.04685591 -0.05614002  0.05111133 -0.04416491 -0.01295188  0.06633321
  0.02150068  0.01328949 -0.02671723 -0.04253641 -0.0198428   0.00500556
 -0.0143372   0.01908546  0.03790782 -0.02720901 -0.03607159 -0.00428732
  0.01140022  0.04330159  0.0226995  -0.03364299 -0.02334993 -0.00048908
 -0.02178952  0.02385082 -0.00028942  0.00100566 -0.03284542  0.00335183
 -0.01953491  0.00039112  0.01302968 -0.00411065 -0.0403197   0.0826552
  0.02162331  0.04560452 -0.05519556  0.06469066 -0.01416294  0.00754313
  0.06652675 -0.01037265  0.05146144  0.03764157 -0.00167219 -0.02792636
 -0.05436959 -0.01833669 -0.04436235  0.01094308 -0.

In [43]:
#printing the vocabulary
for index, word in enumerate(wv.index_to_key):
    if index==10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [44]:
#saving the model
with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

2022-05-29 16:10:53,612 : INFO : Word2Vec lifecycle event {'fname_or_handle': '/var/folders/21/3qr5by0s2yd2xqyy0hshr3kh0000gp/T/gensim-model-wm_okd3k', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-05-29T16:10:53.612865', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'saving'}
2022-05-29 16:10:53,613 : INFO : not storing attribute cum_table
2022-05-29 16:10:53,617 : INFO : saved /var/folders/21/3qr5by0s2yd2xqyy0hshr3kh0000gp/T/gensim-model-wm_okd3k
2022-05-29 16:10:53,618 : INFO : loading Word2Vec object from /var/folders/21/3qr5by0s2yd2xqyy0hshr3kh0000gp/T/gensim-model-wm_okd3k
2022-05-29 16:10:53,621 : INFO : loading wv recursively from /var/folders/21/3qr5by0s2yd2xqyy0hshr3kh0000gp/T/gensim-model-wm_okd3k.wv.* with mmap=None
2022-05-29 16:10:53,622 : INFO : setting ignored attribute cum_table to None
2022-05-29 16:10:53,644 : INFO : Word2Vec lifecycl

In [45]:
#Hyperparameters

In [47]:
#Minimum count 
#Kep on ly those words that has count atleast equal to min_count

In [48]:
model = gensim.models.Word2Vec(sentences, min_count=10)

2022-05-29 16:15:35,331 : INFO : collecting all words and their counts
2022-05-29 16:15:35,334 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-29 16:15:35,434 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-29 16:15:35,435 : INFO : Creating a fresh vocabulary
2022-05-29 16:15:35,441 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 889 unique words (12.734565248531728%% of original 6981, drops 6092)', 'datetime': '2022-05-29T16:15:35.441464', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-29 16:15:35,442 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 43776 word corpus (75.2785802723896%% of original 58152, drops 14376)', 'datetime': '2022-05-29T16:15:35.442341', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.

In [49]:
#length of word vectors
#large vectors results in better model with a cost of 
model = gensim.models.Word2Vec(sentences=sentences, vector_size=200, min_count=5)

2022-05-29 16:19:07,134 : INFO : collecting all words and their counts
2022-05-29 16:19:07,137 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-29 16:19:07,229 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2022-05-29 16:19:07,230 : INFO : Creating a fresh vocabulary
2022-05-29 16:19:07,239 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1750 unique words (25.068041827818366%% of original 6981, drops 5231)', 'datetime': '2022-05-29T16:19:07.239016', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-05-29 16:19:07,240 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 49335 word corpus (84.83801073049938%% of original 58152, drops 8817)', 'datetime': '2022-05-29T16:19:07.239984', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0

In [50]:
#number of workers
#used for training parallelization

In [51]:
model.wv.evaluate_word_analogies(datapath('questions-words.txt'))

2022-05-29 16:22:00,573 : INFO : Evaluating word analogies for top 300000 words in the model on /Users/shri/opt/anaconda3/lib/python3.9/site-packages/gensim/test/test_data/questions-words.txt
2022-05-29 16:22:00,588 : INFO : capital-common-countries: 0.0% (0/6)
2022-05-29 16:22:00,605 : INFO : capital-world: 0.0% (0/2)
2022-05-29 16:22:00,621 : INFO : family: 0.0% (0/6)
2022-05-29 16:22:00,638 : INFO : gram3-comparative: 0.0% (0/20)
2022-05-29 16:22:00,647 : INFO : gram4-superlative: 0.0% (0/12)
2022-05-29 16:22:00,656 : INFO : gram5-present-participle: 0.0% (0/20)
2022-05-29 16:22:00,670 : INFO : gram6-nationality-adjective: 3.3% (1/30)
2022-05-29 16:22:00,682 : INFO : gram7-past-tense: 0.0% (0/20)
2022-05-29 16:22:00,696 : INFO : gram8-plural: 0.0% (0/30)
2022-05-29 16:22:00,701 : INFO : Quadruplets with out-of-vocabulary words: 99.3%
2022-05-29 16:22:00,702 : INFO : NB: analogies containing OOV words were skipped from evaluation! To change this behavior, use "dummy4unknown=True"
202

(0.00684931506849315,
 [{'section': 'capital-common-countries',
   'correct': [],
   'incorrect': [('CANBERRA', 'AUSTRALIA', 'KABUL', 'AFGHANISTAN'),
    ('CANBERRA', 'AUSTRALIA', 'PARIS', 'FRANCE'),
    ('KABUL', 'AFGHANISTAN', 'PARIS', 'FRANCE'),
    ('KABUL', 'AFGHANISTAN', 'CANBERRA', 'AUSTRALIA'),
    ('PARIS', 'FRANCE', 'CANBERRA', 'AUSTRALIA'),
    ('PARIS', 'FRANCE', 'KABUL', 'AFGHANISTAN')]},
  {'section': 'capital-world',
   'correct': [],
   'incorrect': [('CANBERRA', 'AUSTRALIA', 'KABUL', 'AFGHANISTAN'),
    ('KABUL', 'AFGHANISTAN', 'PARIS', 'FRANCE')]},
  {'section': 'currency', 'correct': [], 'incorrect': []},
  {'section': 'city-in-state', 'correct': [], 'incorrect': []},
  {'section': 'family',
   'correct': [],
   'incorrect': [('HE', 'SHE', 'HIS', 'HER'),
    ('HE', 'SHE', 'MAN', 'WOMAN'),
    ('HIS', 'HER', 'MAN', 'WOMAN'),
    ('HIS', 'HER', 'HE', 'SHE'),
    ('MAN', 'WOMAN', 'HE', 'SHE'),
    ('MAN', 'WOMAN', 'HIS', 'HER')]},
  {'section': 'gram1-adjective-to-adver